## 1. Train a PyCaret Model

In this notebook we will train a PyCaret model using the Juice dataset and save the model object as a file, which we will in the second step add to the Snowflake Model Registry.

### Import Libraries

In [1]:
from pycaret.classification import ClassificationExperiment
from pycaret.datasets import get_data

### Train a PyCaret Classification model

Start by running a Classification Experiment using the Juice dataset

In [2]:
data = get_data('juice')

cl_exp = ClassificationExperiment()
cl_exp.setup(data, target='Purchase', session_id=123)
best_model = cl_exp.compare_models()

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


,Description,Value
0,Session id,123
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(749, 19)"
7,Transformed test set shape,"(321, 19)"
8,Ordinal features,1
9,Numeric features,17


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.8278,0.8941,0.7637,0.7909,0.7740,0.6354,0.6388,0.0240
ridge,Ridge Classifier,0.8158,0.0000,0.7431,0.7807,0.7581,0.6099,0.6138,0.0210
lr,Logistic Regression,0.8131,0.8927,0.7192,0.7882,0.7488,0.6010,0.6056,0.3210
gbc,Gradient Boosting Classifier,0.8037,0.8888,0.7499,0.7545,0.7476,0.5877,0.5926,0.0280
lightgbm,Light Gradient Boosting Machine,0.7930,0.8729,0.7257,0.7420,0.7317,0.5636,0.5658,0.0280
ada,Ada Boost Classifier,0.7917,0.8640,0.7018,0.7489,0.7225,0.5565,0.5592,0.0300
xgboost,Extreme Gradient Boosting,0.7851,0.8667,0.7090,0.7377,0.7200,0.5462,0.5496,0.0330
rf,Random Forest Classifier,0.7810,0.8733,0.7090,0.7284,0.7163,0.5385,0.5409,0.0280
et,Extra Trees Classifier,0.7718,0.8482,0.6984,0.7133,0.7038,0.5186,0.5206,0.0360
dt,Decision Tree Classifier,0.7650,0.7582,0.7262,0.6893,0.7056,0.5107,0.5131,0.0230


We can see that the best model from this experience is using *Linear Discriminant Analysis* and if we check **best_model** we can see that it is that model we have.

In [3]:
best_model

LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                           priors=None, shrinkage=None, solver='svd',
                           store_covariance=False, tol=0.0001)

We now have a model that we are satisfied with the performance we can move over to prepare for storing the model in Snowflake using the Snowflake Model Registry.

In order to log the model/experiment into the Snowpark Model Registry we need to 
1) Serialize the model into a file
2) Create a CustomModel class

We will save the model as a file using the **save_model** method

In [4]:
cl_exp.save_model(best_model, "juice_best_model")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/var/folders/bc/j98pnjs533v5106jnkyfvgd00000gn/T/joblib),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['Id', 'WeekofPurchase', 'StoreID',
                                              'PriceCH', 'PriceMM', 'DiscCH',
                                              'DiscMM', 'SpecialCH', 'Sp...
                                                                drop_invariant=False,
                                                                handle_missing='return_nan',
                                                                handle_unknown='value',
                                                                mapping=[{'col': 'Store7',
                                

We have now a trained PyCaret model that is saved to local disc and can go to the next step, where we will create a Custom Model and log it to the Snowflake Model Registry